# API + `jinja2` czyli na ratunek wielu długim ankietom!

Wyobraźmy sobie, że musimy stworzyć bardzo długą ankietę (albo kilka-kilkanaście wersji tej samej ankiety różniącej się szczegółami). Moglibyśmy zrobić to ręcznie tworząc je w LimeSurveyu, ale w przypadku skomplikowanych ankiet, składających się z bardzo wielu pytań, procedura taka naraża nas na ryzyko błędów.

Możemy go uniknąć automatyzując stworzenie ankiet. Spróbujmy napisać skrypt, który tworzyć będzie ankietę z Kwestionariuszem Kodów Moralnych. Kwestionariusz ten opiera się na teorii kodów moralnych autorstwa Grahama i Haidta i można o nim poczytać więcej tutaj: www.moralfoundations.org

Kwestionariusz składa się z dwóch części, które chcemy wyświetlić na osobnych ekranach. Na początek stworzymy "szablon" naszej ankiety za pomocą interfejsu LimeSurveya. Przykładowy szablon mógłby wyglądać tak:

http://kognilab.pl/ls3/index.php/593485?lang=pl

Jak widać w każdej grupie pytań mamy jedno pytanie z instrukcją oraz szablon pytania, w którym treść pytania zastąpiliśmy ciągiem znaków `XXquestiontextXX`. Podobnie kod pytania w LimeSurveyu to teraz `XXquestioncodeXX`. Oznacza to, że łatwo będziemy mogli zastąpić te ciągi znaków za pomocą silnika `jinja2` właściwym kodem i treścią pytania. Wczytajmy najpierw nasze pytania, które mamy zapisane w plikach CSV.

In [27]:
import csv

In [34]:
with open('MFQPART1.csv') as f:
    part1 = list(csv.DictReader(f, delimiter = ';'))
with open('MFQPART2.csv') as f:
    part2 = list(csv.DictReader(f, delimiter = ';'))

In [38]:
part1[:3] # pierwszy trzy pytania

[OrderedDict([('kod', 'EMOTIONALLY'),
              ('pytanie', 'czy ktoś ucierpiał emocjonalnie')]),
 OrderedDict([('kod', 'TREATED'),
              ('pytanie', 'czy ktoś był traktowany inaczej niż inni')]),
 OrderedDict([('kod', 'LOVECOUNTRY'),
              ('pytanie',
               'czy czyjeś działania były przejawem miłości do ojczyzny')])]

Drugą kwestią, którą musimy zrobić, to wyeksportowanie do plików `lss` oraz `lsq` naszej ankiety oraz szablonów pytań. Załóżmy, że to zrobiliśmy i mamy je zapisane na dysku jako `survey_template.lss`, `part1_template.lsq` oraz `part2_template.lsq'.

In [68]:
with open('survey_template.lss') as f:
    survey_template = f.read()
with open('part1_template.lsq') as f:
    part1_template = f.read()
with open('part2_template.lsq') as f:
    part2_template = f.read()

Jak widać pliki wyeksportowane z LimeSurveya to zwykłe pliki w formacie XML. (UWAGA! Musiałem w edytorze tekstowym dodać spacje po i przed `XX` aby działało w jinja2!).

In [70]:
print(part2_template[:1200])

<?xml version="1.0" encoding="UTF-8"?>
<document>
 <LimeSurveyDocType>Question</LimeSurveyDocType>
 <DBVersion>355</DBVersion>
 <languages>
  <language>pl</language>
 </languages>
 <questions>
  <fields>
   <fieldname>qid</fieldname>
   <fieldname>parent_qid</fieldname>
   <fieldname>sid</fieldname>
   <fieldname>gid</fieldname>
   <fieldname>type</fieldname>
   <fieldname>title</fieldname>
   <fieldname>question</fieldname>
   <fieldname>preg</fieldname>
   <fieldname>help</fieldname>
   <fieldname>other</fieldname>
   <fieldname>mandatory</fieldname>
   <fieldname>question_order</fieldname>
   <fieldname>language</fieldname>
   <fieldname>scale_id</fieldname>
   <fieldname>same_default</fieldname>
   <fieldname>relevance</fieldname>
   <fieldname>modulename</fieldname>
  </fields>
  <rows>
   <row>
    <qid><![CDATA[6930]]></qid>
    <parent_qid><![CDATA[0]]></parent_qid>
    <sid><![CDATA[593485]]></sid>
    <gid><![CDATA[2367]]></gid>
    <type><![CDATA[F]]></type>
    <title><![CD

Teraz musimy wykonać za pomocą skryptu następujące czynności:
1. Wrzucić do LimeSurveya nową ankietę
2. Do pierwszej grupy pytań wrzucić pytania z odpowiedniego szablonu wypełnionego danymi
3. Do drugiej grupy pytań wrzucić z pytania z odpowiedniego szablonu wypełnionego danymi

In [44]:
from jinja2 import Template, Environment
from pprint import pprint, pformat
from base64 import b64decode, b64encode
from xmlrpc.client import ServerProxy

Najpierw połączymy się standardowo z LimeSurveyem i stworzymy specjalne środowisko jinja2.

In [59]:
kognilab = ServerProxy('http://kognilab.pl/ls3/index.php/admin/remotecontrol') 
login = 'Użytkownik Testowy'
password = 'test123'
skey = kognilab.get_session_key(login, password)

env = Environment(variable_start_string = 'XX',
                 variable_end_string = 'XX', 
                 block_start_string = 'XV', 
                 block_end_string = 'VX')

Zaimportowanie ankiety jest bardzo proste - wystarczy posłużyć się metodą `import_survey`.

Troche skomplikowanie może wyglądać ciąg `encode` i `decode`. Spróbujmy go przeanalizować.
1. `survey_template.encode()` konwertuje nasz ciag znaków do postaci `bytes` (innej nie przyjmie `b64encode`)
2. `b64encode` konwertuje naszą ankietę do postaci Base64
3. `decode` konwertuje Base64 z powrotem do postaci stringa (innaczej nie wyślemy tego na serwer)

In [72]:
sid = kognilab.import_survey(skey, b64encode(survey_template.encode()).decode(), 'lss')
print(sid)

913779


Teraz kiedy mamy numer ankiety możemy zobaczyć, jakie mamy w ankiecie grupy pytań i zapisać ich numery do zmiennych `gid1` oraz `gid2`.

In [74]:
groups = kognilab.list_groups(skey, sid)
gid1 = groups[0]['gid']
gid2 = groups[1]['gid']
groups

[{'id': {'gid': 2371, 'language': 'pl'},
  'gid': 2371,
  'sid': 913779,
  'group_name': 'PART1',
  'group_order': 1,
  'description': '',
  'language': 'pl',
  'randomization_group': '',
  'grelevance': ''},
 {'id': {'gid': 2372, 'language': 'pl'},
  'gid': 2372,
  'sid': 913779,
  'group_name': 'PART2',
  'group_order': 2,
  'description': '',
  'language': 'pl',
  'randomization_group': '',
  'grelevance': ''}]

Teraz mamy już wszystkie niezbędne informacje - numer nowej ankiety i numery grup w nowej ankiecie. Możemy zaczać wypełniać je pytaniami. Zacznijmy od grupy 1:

In [75]:
template = env.from_string(part1_template)
for question in part1:
    question_text = template.render(questioncode2 = question['kod'],
                                   questiontext = question['pytanie'])
    question_b64 = b64encode(question_text.encode()).decode()
    try:
        kognilab.import_question(skey, # klucz sesji
                                 sid, # numer ankiety
                                 gid1, # grupa do któ©ej chcemy dodać pytanie
                                 question_b64, # pytanie zakodowane w base64
                                 'lsq', # format pytania
                                'Yes') # czy obowiązkowe czy nie
        print('Udało się zaimportować pytanie', question)
    except Exception as e:
        print(e)

Udało się zaimportować pytanie OrderedDict([('kod', 'EMOTIONALLY'), ('pytanie', 'czy ktoś ucierpiał emocjonalnie')])
Udało się zaimportować pytanie OrderedDict([('kod', 'TREATED'), ('pytanie', 'czy ktoś był traktowany inaczej niż inni')])
Udało się zaimportować pytanie OrderedDict([('kod', 'LOVECOUNTRY'), ('pytanie', 'czy czyjeś działania były przejawem miłości do ojczyzny')])
Udało się zaimportować pytanie OrderedDict([('kod', 'RESPECT'), ('pytanie', 'czy ktoś okazał brak szacunku dla władzy')])
Udało się zaimportować pytanie OrderedDict([('kod', 'DECENCY'), ('pytanie', 'czy ktoś pogwałcił zasady czystości i przyzwoitości')])
Udało się zaimportować pytanie OrderedDict([('kod', 'MATH'), ('pytanie', 'czy ktoś jest dobry w rozwiązywaniu zadań matematycznych')])
Udało się zaimportować pytanie OrderedDict([('kod', 'WEAK'), ('pytanie', 'czy ktoś troszczył się o kogoś słabego lub bezbronnego')])
Udało się zaimportować pytanie OrderedDict([('kod', 'UNFAIRLY'), ('pytanie', 'czy ktoś zachował s

To samo zrobimy z drugą częścią:

In [76]:
template = env.from_string(part2_template)
for question in part2:
    question_text = template.render(questioncode = question['kod'],
                                   questiontext = question['pytanie'])
    question_b64 = b64encode(question_text.encode()).decode()
    try:
        kognilab.import_question(skey,
                                 sid,
                                 gid2,
                                 question_b64,
                                 'lsq',
                                 'Yes')
        print('Udało się zaimportować pytanie', question)
    except Exception as e:
        print(e)

Udało się zaimportować pytanie OrderedDict([('kod', 'COMPASSION'), ('pytanie', 'Współczucie dla cierpiących jest najważniejszą cnotą człowieka')])
Udało się zaimportować pytanie OrderedDict([('kod', 'FAIRLY'), ('pytanie', 'Jeśli ustanawia się nowe prawo, to najważniejsze jest, aby wszyscy byli traktowani sprawiedliwie')])
Udało się zaimportować pytanie OrderedDict([('kod', 'HISTORY'), ('pytanie', 'Jestem dumny z historii mojego kraju')])
Udało się zaimportować pytanie OrderedDict([('kod', 'KIDRESPECT'), ('pytanie', 'Szacunek dla władzy i autorytetów jest czymś, czego powinny nauczyć się wszystkie dzieci')])
Udało się zaimportować pytanie OrderedDict([('kod', 'HARMLESSDG'), ('pytanie', 'Nie należy robić rzeczy obrzydliwych, nawet jeśli nikomu nie dzieje się krzywda z tego powodu')])
Udało się zaimportować pytanie OrderedDict([('kod', 'ANIMAL'), ('pytanie', 'Jedną z najgorszych rzeczy, jakie może zrobić człowiek, jest skrzywdzenie bezbronnego zwierzęcia')])
Udało się zaimportować pytanie

Ostatnią rzeczą, którą zrobimy, to aktywujemy przygotowaną ankietę.

In [77]:
kognilab.activate_survey(skey, sid)

{'status': 'OK', 'pluginFeedback': ''}

Teraz możemy się cieszyć naszą nowazaimportowaną ankietą!

http://kognilab.pl/ls3/index.php/913779?lang=pl

Oczywiście w przypadku kwestionariusza kodów moralnych moglibyśmy te 30 pytań dodać ręcznie. W badaniach z semantyki eksperymentalnej nierzadko kwestionariusze mają kilkadziesiat pytań i każdy z nich trzeba przygotować w kilku wersjach - wtedy napisanie skryptu może okazać sie pomocne!

Dodatkową korzyścią jest to, że gdybyśmy chcieli zmienić format pytania albo inny szczegół, to nie musimy edytować 30 pytań - wystarczy, że zmodyfikujemy szablon i ponownie uruchomimy kod.

Ostatnią korzyścią jest korzyść dla reprodukowalności - dzięki użyciu takiej procedury mamy pewność, że pytania w ankiecie, którą wypełnili badani, można prześledzić do prostego pliku CSV.